In [ ]:
# ✅ Environment setup
import os
import pandas as pd
import sqlite3
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent
from sqlalchemy import create_engine

In [ ]:
# 🔐 Load API keys
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
csv_path = '/Users/keshavsaraogi/Desktop/indorama/eureka-data/clean-csv/cleaned_inventory_packaging.csv'
df = pd.read_csv(csv_path)

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.5
)

In [ ]:
csv_agent = create_csv_agent(
    llm,
    csv_path,
    verbose=True,
    allow_dangerous_code=True,
    max_iterations=20,
    max_execution_time=60,
    handle_parsing_errors=True
)

In [ ]:
engine = create_engine("sqlite:///:memory:")
df.to_sql("finance_packaging_data", engine, index=False, if_exists="replace")
db = SQLDatabase(engine)

In [ ]:
# 🧠 Create SQL Agent
sql_agent = create_sql_agent(
    llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=20,
    max_execution_time=60
)

In [ ]:
# 🧭 Routing function
def route_query(query: str):
    query = query.lower()
    if any(keyword in query for keyword in ["sum", "avg", "group", "filter", "division", "unit price"]):
        return "sql"
    else:
        return "csv"

In [ ]:
# 🤖 Master agent logic
def master_agent(query: str):
    source = route_query(query)
    if source == "sql":
        return sql_agent.run(query)
    else:
        return csv_agent.invoke(query)

In [ ]:
# master_agent("Which Company ID has the highest total stock? Give me top 3 companies with their answers.")